In [1]:
import pandas as pd

In [2]:
columns = {"Dte." : np.int64,
          "Nom Dte." : str,
          "Barri" : str,
           "AEB" : int,
           "Any" : int,
           "Població" : int,
           "Naixements Total" : int,
           "Naixements Nens" : int,
           "naixements Nenes" : int,
           "Defuncions Total" : int,
           "Defuncions Homes" : int,
           "Defuncions Dones" : int,
          }

In [39]:
df = pd.DataFrame()


#######################
# add population      #
#######################
add_df = pd.read_csv("evolucio_poblacio_barris.txt", sep=';', decimal=',')
for col in add_df.columns[2:]:
    year = float(col)
    aux_df = add_df[["Dte.", "Barri", col]].copy()
    aux_df.columns = ["Dte.", "Barri", "Població"]
    aux_df.insert(2, "AEB", -1.0)
    aux_df.insert(3, "Any", year)
    df = aux_df.append(df, ignore_index=True)
    def correctDecimalPlaces(item):
        if item > 100.0:
            return item
        else:
            return item*1000
    df["Població"] = df["Població"].astype(float).apply(correctDecimalPlaces)
    
    
#######################
# add natality        #
#######################
df2 = pd.DataFrame()
add_df = pd.read_csv("evolucio_natalitat_bar.txt", sep=';', decimal=',')
for col in add_df.columns[2:]:
    if (col.split()[1] != "Total"):
        continue
    year = col.split()[0]
    aux_df = add_df[["Dte.", "Barri", col, "{} Nens".format(year), "{} Nenes".format(year)]].copy()
    aux_df.columns = ["Dte.", "Barri", "Naixements Total", "Naixements Nens", "Naixements Nenes"]
    aux_df["Any"] = float(year)
    aux_df["AEB"] = -1.0
    df2 = aux_df.append(df2, ignore_index=True)
df = pd.merge(df, df2, how='outer')


#######################
# add mortality       #
#######################
df2 = pd.DataFrame()
add_df = pd.read_csv("evolucio_defuncions_bar.txt", sep=';', decimal=',')
for col in add_df.columns[2:]:
    if (col.split()[1] != "Total"):
        continue
    year = col.split()[0]
    aux_df = add_df[["Dte.", "Barri", col, "{} Homes".format(year), "{} Dones".format(year)]].copy()
    aux_df.columns = ["Dte.", "Barri", "Defuncions Total", "Defuncions Homes", "Defuncions Dones"]
    aux_df["Any"] = float(year)
    aux_df["AEB"] = -1.0
    df2 = aux_df.append(df2, ignore_index=True)
df = pd.merge(df, df2, how='outer')


#######################
# add studies         #
#######################
df2 = pd.DataFrame()
add_df = pd.read_csv("nivell_estudis_barri.csv", sep=";", skiprows=4, decimal=',')
# fix encoding
barris = {barri.split(".")[0] : barri for barri in np.unique(df["Barri"])}
def FixNeighbourhoodNames(item, barris):
    return barris.get(item.split(".")[0], np.nan)
add_df["Territori"] = add_df["Territori"].apply(FixNeighbourhoodNames, args=(barris,))
for col in add_df.columns[3:-1]:
    if col.split()[-2] != "estudis":
        continue
    year = col.split()[-1]
    aux_df = add_df[["Territori", col, "{} amb estudis obligatoris {} ".format(col[:13], year),
                     "{} amb batxillerat superior, CFGM {} ".format(col[:13], year),
                     "{} amb estudis universitaris i CFGS {} ".format(col[:13], year)]].copy()
    aux_df.columns = ["Barri", "% Població sense estudis", "% Població amb estudis obligatoris",
                      "% Població amb batxillerat superior o CFGM", "% Població amb estudis universitaris o CFGS"]
    aux_df["Any"] = float(year)
    aux_df["AEB"] = -1.0
    df2 = aux_df.append(df2, ignore_index=True)
df = pd.merge(df, df2, how='outer')


########################
# add District name    #
########################
districtes = {1 : "Ciutat Vella",
              2 : "Eixample",
              3 : "Sants-Montjuïc",
              4 : "Les Corts",
              5 : "Sarrià-Sant Gervasi",
              6 : "Gràcia",
              7 : "Horta-Guinardó",
              8 : "Nou Barris",
              9 : "Sant Andreu",
              10 : "Sant Martí"}
def findDistricName(item, districtes):
    dist_number = item
    return districtes.get(dist_number, np.nan)
df.insert(1, "Nom Dte.", df["Dte."].apply(findDistricName, args=(districtes,)))


#######################
# format coulmns      #
#######################
df["AEB"] = df["AEB"].astype(int)
df["Dte."] = df["Dte."].astype(int)
df["Any"] = df["Any"].astype(int)
df["Població"] = df["Població"].astype(float)


#######################
# save formatted data #
#######################
df.to_csv("ajuntament_bcn_formatted_data.csv", sep=';', encoding='utf-8', index=False, decimal=',')

In [4]:
df

,Dte.,Nom Dte.,Barri,AEB,Any,Població,Naixements Total,Naixements Nens,Naixements Nenes,Defuncions Total,Defuncions Homes,Defuncions Dones,% Població sense estudis,% Població amb estudis obligatoris,% Població amb batxillerat superior o CFGM,% Població amb estudis universitaris o CFGS
0,1,Ciutat Vella,1. el Raval,-1,2016,47129.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,55.1,18.0,22.7
1,1,Ciutat Vella,2. el Barri Gòtic,-1,2016,15614.0,NaN,NaN,NaN,NaN,NaN,NaN,2.3,34.9,23.9,38.1
2,1,Ciutat Vella,3. la Barceloneta,-1,2016,15021.0,NaN,NaN,NaN,NaN,NaN,NaN,4.7,48.1,20.5,25.6
3,1,Ciutat Vella,"4. Sant Pere, Santa Caterina i la Ribera",-1,2016,22306.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,35.9,22.6,37.7
4,2,Eixample,5. el Fort Pienc,-1,2016,31653.0,NaN,NaN,NaN,NaN,NaN,NaN,1.9,32.5,27.2,37.0
5,2,Eixample,6. la Sagrada Família,-1,2016,51322.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,34.9,26.4,35.0
6,2,Eixample,7. la Dreta de l'Eixample,-1,2016,43644.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3,21.6,26.2,49.5
7,2,Eixample,8. l'Antiga Esquerra de l'Eixample,-1,2016,41854.0,NaN,NaN,NaN,NaN,NaN,NaN,1.6,24.5,27.3,45.4
8,2,Eixample,9. la Nova Esquerra de l'Eixample,-1,2016,57648.0,NaN,NaN,NaN,NaN,NaN,NaN,1.7,29.7,27.3,40.1
9,2,Eixample,10. Sant Antoni,-1,2016,38184.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,36.2,26.0,34.0


In [38]:
df.dtypes

Dte.                                             int64
Nom Dte.                                        object
Barri                                           object
AEB                                              int64
Any                                              int64
Població                                       float64
Naixements Total                               float64
Naixements Nens                                float64
Naixements Nenes                               float64
Defuncions Total                               float64
Defuncions Homes                               float64
Defuncions Dones                               float64
% Població sense estudis                       float64
% Població amb estudis obligatoris             float64
% Població amb batxillerat superior o CFGM     float64
% Població amb estudis universitaris o CFGS    float64
dtype: object

In [24]:
add_df = pd.read_csv("nivell_estudis_barri.csv", sep=";", skiprows=4, decimal=',')

TypeError: data type not understood

In [21]:
df.columns

Index([u'Dte.', u'Barri', u'AEB', u'Any', u'Població'], dtype='object')

In [38]:

import locale
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')
add_df = pd.read_csv("evolucio_poblacio_barris.txt", sep=';')
add_df["2010"] = add_df["2010"].astype(float)
add_df[add_df["Dte."] == 7]


,Dte.,Barri,2010,2011,2012,2013,2014,2015,2016
32,7,33. el Baix Guinardó,26.022,26.022,25.918,25.641,25.527,25.466,25.547
33,7,34. Can Baró,9.119,8.928,9.005,8.969,8.859,8.819,8.965
34,7,35. el Guinardó,35.655,35.687,35.664,35.679,35.534,35.773,36.136
35,7,36. la Font d'en Fargues,9.588,9.530,9.569,9.490,9.440,9.407,9.400
36,7,37. el Carmel,32.349,32.108,32.012,31.789,31.659,31.456,31.355
37,7,38. la Teixonera,11.821,11.796,11.776,11.654,11.435,11.361,11.277
38,7,39. Sant Genís dels Agudells,7.159,7.035,6.985,6.935,6.884,6.754,6.725
39,7,40. Montbau,5.256,5.211,5.144,5.111,5.047,5.046,5.053
40,7,41. la Vall d'Hebron,5.611,5.580,5.569,5.494,5.490,5.468,5.686
41,7,42. la Clota,453.000,462.000,452.000,438.000,521.000,542.000,583.000


'ca_ES.UTF-8'